In [1]:
import sys
sys.path.append('..')

# Using nbtlib

The Named Binary Tag (NBT) file format is a simple structured binary format that is mainly used by the game Minecraft (see the [official specification](http://wiki.vg/NBT) for more details). This short documentation will show you how you can manipulate nbt data using the `nbtlib` module.

## Loading a file

In [2]:
from nbtlib import nbt

nbt_file = nbt.load('nbt_files/bigtest.nbt')
nbt_file.root['stringTest']

String('HELLO WORLD THIS IS A TEST STRING ÅÄÖ!')

By default `nbt.load` will figure out by itself if the specified file is gzipped, but you can also use the `gzipped=` keyword only argument if you know in advance whether the file is gzipped or not.

In [3]:
uncompressed_file = nbt.load('nbt_files/hello_world.nbt', gzipped=False)
uncompressed_file.gzipped

False

You can also read nbt data directly from an already opened file-like object with the `.parse` class method.

In [4]:
from nbtlib import File

with open('nbt_files/hello_world.nbt', 'rb') as f:
    hello_world = File.parse(f)
hello_world

<File 'hello world': Compound({'name': String('Bananrama')})>

## Accessing file data

The `File` class inherits from `Compound`, which inherits from `dict`. This means that you can use standard `dict` operations to access data inside of the file. As most files usually contain a single root tag, there is a shorthand to access it directly.

In [5]:
nbt_file.keys()

dict_keys(['Level'])

In [6]:
nbt_file.root == nbt_file['Level']

True

## Modifying files

In [7]:
from nbtlib.tag import *

with nbt.load('nbt_files/demo.nbt') as demo:
    demo.root['counter'] = Int(demo.root['counter'] + 1)

If you don't want to use a context manager, you can call the `.save` method manually to overwrite the original file or make a copy by specifying a different path. The `.save` method also accepts the `gzipped=` keyword only argument. By default, the copy will be gzipped if the original file is gzipped.

In [8]:
demo = nbt.load('nbt_files/demo.nbt')
...
demo.save()  # overwrite
demo.save('nbt_files/demo_copy.nbt', gzipped=True)  # make a gzipped copy

nbt.load('nbt_files/demo_copy.nbt').root['counter']

Int(32)

You can also write nbt data to an already opened file-like object using the `.write` method.

In [9]:
with open('nbt_files/demo_copy.nbt', 'wb') as f:
    demo.write(f)

## Creating files

In [10]:
new_file = File({
    '': Compound({
        'foo': String('bar'),
        'spam': IntArray([1, 2, 3]),
        'egg': List[String](['hello', 'world'])
    })
})
new_file.save('nbt_files/new_file.nbt')

nbt.load('nbt_files/new_file.nbt').gzipped

False

New files are uncompressed by default.

## Performing operations on tags

All nbt tags defined in the library inherit from the associated python builtin type.

| Python builtin type | Associated nbt tags             |
| ------------------- | ------------------------------- |
| **int**             | `Byte`, `Short`, `Int`, `Long`  |
| **float**           | `Float`, `Double`               |
| **str**             | `String`                        |
| **array**           | `ByteArray`, `IntArray`         |
| **list**            | `List`                          |
| **dict**            | `Compound`                      |

All the methods and operations that are usually available on the the base python builtin types can be used on the associated tags.

In [11]:
my_list = List[String](char.upper() for char in 'hello')
my_list.reverse()
my_list[3:]

[String('E'), String('H')]

In [12]:
my_pizza = Compound({
    'name':String('Margherita'),
    'price': Double(5.7),
    'size': String('medium')
})

my_pizza.update({'name': String('Calzone'), 'size': String('large')})
my_pizza['price'] = Double(my_pizza['price'] + 2.5)
my_pizza

Compound({'name': String('Calzone'), 'price': Double(8.2), 'size': String('large')})

Note that while using `repr()` on nbt tags outputs a python representation of the tag, calling `str()` on nbt tags (or simply printing them) will return the nbt literal representing that tag.

In [13]:
str(my_pizza)

'{name:Calzone,price:8.2d,size:large}'

In [14]:
print(Compound({
    'numbers': IntArray([1, 2, 3]), 
    'foo': String('bar'),
    'syntax breaking': Float(42),
    'spam': String('{"text":"Hello, world!\nThis is a new line."}')
}))

{numbers:[I;1,2,3],foo:bar,"syntax breaking":42.0f,spam:"{\"text\":\"Hello, world!\\nThis is a new line.\"}"}


## Creating tags from nbt literals

You can also use the literal notation of nbt data to create tags.

In [15]:
from nbtlib import parse_nbt

parse_nbt('hello')

String('hello')

In [16]:
parse_nbt('{foo:[{bar:[I;1,2,3]},{spam:6.7f}]}')

Compound({'foo': List[Compound]([Compound({'bar': IntArray([1, 2, 3])}), Compound({'spam': Float(6.7)})])})

## Defining schemas

In order to avoid wrapping values manually every time you edit a compound tag, you can define a schema that will take care of converting python types to predefined nbt tags automatically.

In [17]:
from nbtlib import schema

MySchema = schema('MySchema', {
    'foo': String, 
    'bar': Short
})

my_object = MySchema({'foo': 'hello world', 'bar': 21})
my_object['bar'] *= 2
my_object

MySchema({'foo': String('hello world'), 'bar': Short(42)})

By default, you can interact with keys that are not defined in the schema. However, if you use the `strict=` keyword only argument, the schema instance will raise a `TypeError` whenever you try to access a key that wasn't defined in the original schema.

In [18]:
MyStrictSchema = schema('MyStrictSchema', {
    'foo': String,
    'bar': Short
}, strict=True)

strict_instance = MyStrictSchema()
strict_instance.update({'foo': 'hello world'})
strict_instance

MyStrictSchema({'foo': String('hello world')})

In [19]:
try:
    strict_instance['something'] = List[String](['this', 'raises', 'an', 'error'])
except TypeError as exc:
    print(exc)

Invalid key 'something'


The `schema` function is a helper that creates a class that inherits from `CompoundSchema`. This means that you can also inherit from the class manually.

In [20]:
from nbtlib import CompoundSchema

class MySchema(CompoundSchema):
    schema = {
        'foo': String, 
        'bar': Short
    }

MySchema({'foo': 'hello world', 'bar': 42})

MySchema({'foo': String('hello world'), 'bar': Short(42)})

You can also set the `strict` class attribute to `True` to create a strict schema type.

In [21]:
class MyStrictSchema(CompoundSchema):
    schema = {
        'foo': String, 
        'bar': Short
    }
    strict = True

try:
    MyStrictSchema({'something': Byte(5)})
except TypeError as exc:
    print(exc)

Invalid key 'something'


## Combining schemas and custom file types

If you need to deal with files that always have a particular structure, you can create a specialized file type by combining it with a schema. For instance, this is how you would create a file type that opens [minecraft structure files](https://minecraft.gamepedia.com/Structure_block_file_format).

First, we need to define what a minecraft structure is, so we create a schema that matches the tag hierarchy.

In [22]:
Structure = schema('Structure', {
    'DataVersion': Int,
    'author': String,
    'size': List[Int],
    'palette': List[schema('State', {
        'Name': String,
        'Properties': Compound,
    })],
    'blocks': List[schema('Block', {
        'state': Int,
        'pos': List[Int],
        'nbt': Compound,
    })],
    'entities': List[schema('Entity', {
        'pos': List[Double],
        'blockPos': List[Int],
        'nbt': Compound,
    })],
})

Now let's test our schema by creating a structure. We can see that all the types are automatically applied.

In [23]:
new_structure = Structure({
    'DataVersion': 1139,
    'author': 'dinnerbone',
    'size': [1, 2, 1],
    'palette': [
        {'Name': 'minecraft:dirt'}
    ],
    'blocks': [
        {'pos': [0, 0, 0], 'state': 0},
        {'pos': [0, 1, 0], 'state': 0}
    ],
    'entities': [],
})

type(new_structure['blocks'][0]['pos'])

nbtlib.tag.List[Int]

In [24]:
type(new_structure['entities'])

nbtlib.tag.List[Entity]

Now we can create a custom file type that wraps our structure schema. Since structure files are always gzipped we can override the load method to default the `gzipped` argument to `True`. We also overwrite the constructor so that it can take directly an instance of our structure schema as argument.

In [25]:
class StructureFile(File, schema('StructureFileSchema', {'': Structure})):
    def __init__(self, structure_data=None):
        super().__init__({'': structure_data or {}})
        self.gzipped = True
    @classmethod
    def load(cls, filename, gzipped=True):
        return super().load(filename, gzipped)

We can now use the custom file type to load, edit and save structure files without having to specify the tags manually.

In [26]:
structure_file = StructureFile(new_structure)
structure_file.save('nbt_files/new_structure.nbt')  # you can load it in a minecraft world!

So now let's try to edit the structure. We're going to replace all the dirt blocks with stone blocks.

In [27]:
structure_file = StructureFile.load('nbt_files/new_structure.nbt')
structure_file.root['palette'][0]['Name'] = 'minecraft:stone'
structure_file.save()

As you can see we didn't need to specify any tag to edit the file.